### [117 填充每个节点的下一个右侧节点指针II](https://leetcode.cn/problems/populating-next-right-pointers-in-each-node-ii/description/?envType=study-plan-v2&envId=top-interview-150)  
<font color=#FFCC33>中等</font>

首先应该想到二叉树的`层序遍历`方法: 利用队列push每层的左右子树，然后循环每层的节点数(第一层一个，第二层两个，第n层n个)，就是层序遍历  

In [9]:
from collections import deque, defaultdict


class Node:
    def __init__(self, val: int = 0, left: 'Node' = None, right: 'Node' = None, next: 'Node' = None):
        self.val = val
        self.left = left
        self.right = right
        self.next = next


In [17]:
def layerVisit(root: 'Node'):
    queue = deque()
    queue.append(root)

    layer = 1
    while queue:
        for i in range(layer):
            node = queue.popleft()
            print(node.val, end='')
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)

        print('#', end='')
        layer += 1

In [18]:
if __name__ == '__main__':
    root = Node(val=1, left=Node(val=2, left=Node(val=4), right=Node(val=5)), right=Node(val=3, right=Node(val=7)))
    layerVisit(root)

1#23#457#

在做层序遍历时，也可以每次用队列的长度来做循环，这样可以节约一些空循环

In [3]:
def layerVisit2(root: 'Node'):
    queue: deque = deque()
    queue.append(root)

    while queue:
        loop_len = len(queue)
        for i in range(loop_len):
            node = queue.popleft()
            print(node.val, end='')
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)

        print('#', end='')


if __name__ == '__main__':
    root = Node(val=1, left=Node(val=2, left=Node(val=4), right=Node(val=5)), right=Node(val=3, right=Node(val=7)))
    layerVisit2(root)

1#23#457#

层序遍历，也可以用列表而不是队列:

In [4]:
def layerVisit3(root: 'Node'):
    list = [root]

    while list:
        tmp_list = list
        list = []
        for node in tmp_list:
            print(node.val, end='')
            if node.left:
                list.append(node.left)
            if node.right:
                list.append(node.right)

        print('#', end='')


if __name__ == '__main__':
    root = Node(val=1, left=Node(val=2, left=Node(val=4), right=Node(val=5)), right=Node(val=3, right=Node(val=7)))
    layerVisit3(root)

1#23#457#

层序遍历，除了用BFS以外，也可以用DFS。在DFS的过程中按照层数来把每个节点的值记录到对应层的数组里:  
在`Python`中，字典（dict）从`Python 3.7`版本开始，已经正式成为有序集合。它会严格按照键值对插入的先后来维护顺序

In [13]:
from collections import defaultdict

def layerVisit4(root: 'Node'):
    level_nums = defaultdict(list)
    level = 1

    def dfs(node: 'Node', level: int):
        if node is None:
            return
        if level_nums[level]:
            level_nums[level].append(node.val)
        else:
            level_nums[level] = [node.val]
        dfs(node.left, level + 1)
        dfs(node.right, level + 1)

    dfs(root, level)
    for li in level_nums:
        for num in level_nums[li]:
            print(num, end='')
        print('#', end='')


if __name__ == '__main__':
    root = Node(val=1, left=Node(val=2, left=Node(val=4), right=Node(val=5)), right=Node(val=3, right=Node(val=7)))
    layerVisit4(root)

1#23#457#

回到本题，需要在做层序遍历的时候，把每层的用next指针连起来, 还是使用层序遍历的方法:

In [ ]:
class Solution:
    def connect(self, root: 'Node') -> 'Node':
        queue = deque()
        if root is None:
            return None

        queue.append(root)

        while queue:
            loop_len = len(queue)
            before_node = None

            for i in range(loop_len):
                node = queue.popleft()
                if before_node:
                    before_node.next = node
                before_node = node

                if node.left:
                    queue.append(node.left)
                if node.right:
                    queue.append(node.right)
        return root


但这个解法的空间复杂度不是常量级别的，和二叉树的最大层的节点个数相关。  
考虑一种常量级别的解法：  
仔细思考，我们之所以需要队列来储存节点，是因为我们做层序遍历的时候，需要在队列里保存每一层节点的前后关系。<font color=#FF3366>换个角度思考，保存节点的前后关系，一定要用队列吗？使用链表指针一样可以维护节点的顺序关系。</font>  
用链表关系替换上面层序遍历的队列，就是本题的答案。
本题中的next指针，天然的连接着每一层。  
因此只要我们能迭代每一层的开头的节点，就能知道每一层所有的节点(通过next指针访问)  


正确答案: 三指针方案

In [ ]:
class Solution:
    def connect(self, root: 'Node') -> 'Node':
        if root is None:
            return None

        cur = root
        while cur:
            next_head = None
            next_tail = None
            node = cur
            
            while node:
                if node.left:
                    if next_tail:
                        next_tail.next = node.left
                    next_tail = node.left
                    if next_head is None:
                        next_head = node.left

                if node.right:
                    if next_tail:
                        next_tail.next = node.right
                    next_tail = node.right
                    if next_head is None:
                        next_head = node.right
                    
                node = node.next
            cur = next_head
        return root

使用哨兵节点简化判断

In [ ]:
class Solution:
    def connect(self, root: 'Node') -> 'Node':
        cur = root
        while cur:
            nxt = dummy = Node()  # 下一层的链表
            while cur:  # 遍历当前层的链表
                if cur.left:
                    nxt.next = cur.left  # 下一层的相邻节点连起来
                    nxt = cur.left
                if cur.right:
                    nxt.next = cur.right  # 下一层的相邻节点连起来
                    nxt = cur.right
                cur = cur.next  # 当前层链表的下一个节点
            cur = dummy.next  # 下一层链表的头节点
        return root